
# Left and Right Inverses of a Matrix 



## Quick definitions
- A **left inverse** of $A$ is a matrix $L$ such that $LA = I$. It exists when $A$ has **full column rank** (e.g., a *tall* matrix, more rows than columns).
- A **right inverse** of $A$ is a matrix $R$ such that $AR = I$. It exists when $A$ has **full row rank** (e.g., a *wide* matrix, more columns than rows).
- When $A$ is square and invertible, $A^{-1}$ is both a left and a right inverse.

**Least squares (tall matrices):** For full column rank, the canonical left inverse is $L = (A^\top A)^{-1}A^\top$. Then the least‑squares solution to $Ax = b$ is $x = Lb$, and $Ax$ is the orthogonal **projection of $b$ onto Col(A)**.

**Minimum norm (wide matrices):** For full row rank, a right inverse is $R = A^\top(AA^\top)^{-1}$. Then one solution of $Ax = b$ is $x = Rb$. 


 


When to prefer computing the right inverse directly
1) The matrix is wide $(m≪n)$ and full row rank
If A is $m\times n$ with $m≪n$, computing the right inverse via
$R=A^⊤(AA^⊤)^{−1}$
requires inverting only an $m\times m$ matrix $AA^⊤$, which is much cheaper than anything that touches an $n \times n$ object.




Example scenario:
$m=100$, $n=100,000$.

$AA^⊤$ is $100\times 100$ →  tractable.
$A^⊤A$ is $100,000\times 100,000$ → infeasible.


## Setup
We will use only NumPy to keep things lightweight. Utility helpers are included to check identities and orthogonality numerically (within a tolerance).


In [1]:

import numpy as np

def is_close(A, B, tol=1e-9):
    return np.allclose(A, B, atol=tol, rtol=0)

def print_matrix(name, M):
    print(f"{name} shape={M.shape} {M} ")

def orthogonality_check(A, r, tol=1e-9):
    v = A.T @ r
    print(f"||A^T r||_2 = {np.linalg.norm(v):.3e} (should be ~ 0 if residual is orthogonal to col(A)) ")

np.set_printoptions(precision=4, suppress=True)



## 1) Tall matrix (full column rank) → Left inverse & least squares geometry
**Theory:** If $A\in\mathbb{R}^{m\times n}$ with $m>n$ and full column rank, then $L=(A^\top A)^{-1}A^\top$ satisfies $LA=I_n$. For any $b\in\mathbb{R}^m$, the least‑squares solution is $x=Lb$. The fitted vector $\hat b=Ax$ is the **orthogonal projection** of $b$ onto $\\operatorname{col}(A)$, and the residual $r=b-\hat b$ is orthogonal to that space (i.e., $A^\top r=0$).


In [2]:

# A simple tall matrix (3x2) with full column rank
A_tall = np.array([[1., 2.],
                   [3., 4.],
                   [5., 7.]])

# Left inverse via normal equations (for didactic purposes)
L = np.linalg.inv(A_tall.T @ A_tall) @ A_tall.T

print_matrix('A_tall', A_tall)
print_matrix('Left inverse L = (A^T A)^{-1} A^T', L)
print('Check L A = I: ', is_close(L @ A_tall, np.eye(A_tall.shape[1])))

# Least squares for Ax = b
b = np.array([2., 1., -1.])
x_ls = L @ b
b_hat = A_tall @ x_ls
r = b - b_hat

print_matrix('b', b)
print_matrix('x_ls (least squares)', x_ls)
print_matrix('b_hat = A x_ls (projection of b onto col(A))', b_hat)
print_matrix('Residual r = b - b_hat', r)
orthogonality_check(A_tall, r)


A_tall shape=(3, 2) [[1. 2.]
 [3. 4.]
 [5. 7.]] 
Left inverse L = (A^T A)^{-1} A^T shape=(2, 3) [[-2.0714  0.7857  0.1429]
 [ 1.5    -0.5     0.    ]] 
Check L A = I:  True
b shape=(3,) [ 2.  1. -1.] 
x_ls (least squares) shape=(2,) [-3.5  2.5] 
b_hat = A x_ls (projection of b onto col(A)) shape=(3,) [ 1.5 -0.5 -0. ] 
Residual r = b - b_hat shape=(3,) [ 0.5  1.5 -1. ] 
||A^T r||_2 = 8.967e-14 (should be ~ 0 if residual is orthogonal to col(A)) 



## 2) Wide matrix (full row rank) → Right inverse & families of solutions
**Theory:** If $A\in\mathbb{R}^{m\times n}$ with $m<n$ and full row rank, then $R=A^\top(AA^\top)^{-1}$ satisfies $AR=I_m$. Given $b\in\mathbb{R}^m$, one solution to $Ax=b$ is $x=Rb$. In fact, **all** solutions are $x=Rb+z$ for any $z\in\\mathcal{N}(A)$ (the nullspace). The pseudoinverse $A^+$ returns the **minimum‑norm** solution among these.


In [3]:

# A simple wide matrix (2x3) with full row rank
A_wide = np.array([[1., 2., 3.],
                  [2., 1., 0.]])

# Right inverse via normal equations on rows (for didactic purposes)
R = A_wide.T @ np.linalg.inv(A_wide @ A_wide.T)

print_matrix('A_wide', A_wide)
print_matrix('Right inverse R = A^T (A A^T)^{-1}', R)
print('Check A R = I: ', is_close(A_wide @ R, np.eye(A_wide.shape[0])))

# Solve Ax = b
b2 = np.array([1., -1.])
x0 = R @ b2
print_matrix('b2', b2)
print_matrix('One solution x0 = R b2', x0)
print_matrix('A x0 (should equal b2)', A_wide @ x0)

# Nullspace via SVD: right singular vectors corresponding to ~zero singular values
U, s, Vt = np.linalg.svd(A_wide, full_matrices=True)
# For a 2x3 full row rank matrix, nullity = 1; the null vector is the last row of Vt (since s sorted desc)
nvec = Vt[-1, :].reshape(-1, 1)  # (3,1)
# Verify it's in the nullspace
print_matrix('A_wide @ nvec (should be ~0)', A_wide @ nvec)
# Family of solutions: x = x0 + nvec * t
t = 3.0
x_family = x0.reshape(-1,1) + nvec * t
print_matrix('x_family (t=3)', x_family)
print_matrix('A x_family (should equal b2)', A_wide @ x_family)


A_wide shape=(2, 3) [[1. 2. 3.]
 [2. 1. 0.]] 
Right inverse R = A^T (A A^T)^{-1} shape=(3, 2) [[-0.0556  0.4444]
 [ 0.1111  0.1111]
 [ 0.2778 -0.2222]] 
Check A R = I:  True
b2 shape=(2,) [ 1. -1.] 
One solution x0 = R b2 shape=(3,) [-0.5  0.   0.5] 
A x0 (should equal b2) shape=(2,) [ 1. -1.] 
A_wide @ nvec (should be ~0) shape=(2, 1) [[0.]
 [0.]] 
x_family (t=3) shape=(3, 1) [[ 0.7247]
 [-2.4495]
 [ 1.7247]] 
A x_family (should equal b2) shape=(2, 1) [[ 1.]
 [-1.]] 



## 3) The transpose trick
**Key idea:** If $A$ is tall full‑column‑rank and you computed the left inverse $L=(A^\top A)^{-1}A^\top$, then for **the transposed problem** $A^\top$ (which is wide full‑row‑rank), the matrix **$L^\top$ is a right inverse of $A^\top$**, because

$$
(A^\top)\,L^\top = (LA)^\top = I^\top = I.
$$

This is handy if an algorithm needs both forms but building only one is cheaper or already available.


In [4]:

# Verify the transpose trick with our tall example
A = A_tall
L = np.linalg.inv(A.T @ A) @ A.T
check = A.T @ L.T
print_matrix('(A^T) * (L^T)', check)
print('Is it identity?', is_close(check, np.eye(A.shape[1])))


(A^T) * (L^T) shape=(2, 2) [[1. 0.]
 [0. 1.]] 
Is it identity? True


Exercises:
1. For a large tall matrix, find right inverse using left inverse (Hint: Use transpose idea). Verify your answer. 
Give an explicit illustration with $3 \times 2$ matrix.
2. For a large wide matrix, find left inverse using right invese. Verify your answer. Give an explicit illustration with $2 \times 3$ matrix.

# SOLUTIONS

## Exercise 1: Tall matrix (left inverse ⇒ right inverse)

Suppose $A$ is **tall**, full column rank, size $3 \times 2$. Then $L=(A^\top A)^{-1}A^\top$ is its **left inverse**, i.e. $LA=I_2$.
Transpose trick says:

$$
(A^\top) L^\top = (LA)^\top = I.
$$

So $L^\top$ is a **right inverse** of $A^\top$.

In [53]:
import numpy as np


A_tall = np.array([[1., 2.],
                   [3., 4.],
                   [5., 7.]]) 

L = np.linalg.inv(A_tall.T @ A_tall) @ A_tall.T
R_of_AT = L.T

print("For small example:")
print("\tCheck (A^T)(L^T) = I :")
print("\t", str(A_tall.T @ R_of_AT).replace("\n", "\n\t"))
print("\tIs close to identity?", is_close(A_tall.T @ R_of_AT, np.eye(A_tall.shape[1])))

m, n = 2000, 50 
A_tall = np.random.randn(m, n)
print(A_tall)
print(A_tall.shape)
L = np.linalg.inv(A_tall.T @ A_tall) @ A_tall.T
L_rounded= np.round(L,6)
R_of_AT = L.T

print("\nL of A transpose:\n",L_rounded) 
print("Tall matrix test (with big data):")
print("\t", str(A_tall.T @ R_of_AT).replace("\n", "\n\t"))

print("\tShape A:", A_tall.shape)
print("\tCheck (A^T)(L^T) ≈ I:", is_close(A_tall.T @ R_of_AT, np.eye(n)))

For small example:
	Check (A^T)(L^T) = I :
	 [[1. 0.]
	 [0. 1.]]
	Is close to identity? True
[[-0.082507 -1.219388 -0.485696 ... -0.73513   1.166367 -0.699473]
 [-0.034351  0.968975  0.800277 ...  0.56237  -0.94788   0.572681]
 [-0.821311  0.436029  0.426729 ...  1.164117  0.57548  -0.493805]
 ...
 [ 0.343064 -1.667508  0.049296 ...  0.120757 -1.95168   0.880126]
 [-0.670258  0.515984 -0.240789 ...  0.020652  0.175235 -1.771702]
 [ 0.28527   0.714962 -0.593721 ...  1.998433  0.068528  0.759196]]
(2000, 50)

L of A transpose:
 [[-0.000065 -0.000044 -0.000492 ...  0.000131 -0.000408  0.000227]
 [-0.000663  0.000334  0.000154 ... -0.000807  0.000266  0.000483]
 [-0.000354  0.000421  0.000211 ... -0.000025 -0.000143 -0.000339]
 ...
 [-0.000303  0.000231  0.000595 ...  0.000021  0.000057  0.000936]
 [ 0.000602 -0.000492  0.000326 ... -0.001081  0.000067  0.000028]
 [-0.000317  0.000291 -0.000263 ...  0.00034  -0.000978  0.000273]]
Tall matrix test (with big data):
	 [[ 1.  0.  0. ...  0.  0


$$
L = (A^T A)^{-1} A^T, 
\quad
A = \begin{bmatrix}
1 & 2 \\
3 & 4 \\
5 & 7
\end{bmatrix}.
$$

---


$$
A^T A =
\begin{bmatrix}
35 & 49 \\
49 & 69
\end{bmatrix}.
$$

---


$$
(A^T A)^{-1} 
= \tfrac{1}{14}
\begin{bmatrix}
69 & -49 \\
-49 & 35
\end{bmatrix}.
$$

---


$$
L = \tfrac{1}{14}
\begin{bmatrix}
69 & -49 \\
-49 & 35
\end{bmatrix}
\begin{bmatrix}
1 & 3 & 5 \\
2 & 4 & 7
\end{bmatrix}.
$$

$$
L = 
\begin{bmatrix}
-29/14 & 11/14 & 1/7 \\
3/2 & -1/2 & 0
\end{bmatrix}.
$$

---


$$
LA = 
\begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}.
$$


Verification: `(A^T)(L^T) ≈ I₂`.
So indeed, the transpose of the left inverse is a right inverse for $A^\top$.

## Exercise 2: Wide matrix (right inverse ⇒ left inverse)

Now $A$ is **wide**, full row rank, size $2 \times 3$. Then

$$
R = A^\top (AA^\top)^{-1}
$$

is a **right inverse**, i.e. $AR=I_2$.
Transpose trick gives:

$$
R^\top (A^\top) = (AR)^\top = I.
$$

So $R^\top$ is a **left inverse** of $A^\top$.


In [56]:
A_wide = np.array([[1., 2., 3.],
                   [2., 1., 0.]]) 

R = A_wide.T @ np.linalg.inv(A_wide @ A_wide.T)
L_of_AT = R.T

print("For small example:")
print("\tCheck (R^T)(A^T) = I :")
print("\t", str(L_of_AT @ A_wide.T).replace("\n", "\n\t"))
print("\tIs close to identity?", is_close(L_of_AT @ A_wide.T, np.eye(A_wide.shape[0])))

m, n = 50, 2000
A_wide = np.random.randn(m, n)
print(A_wide)
print(A_wide.shape)
R = A_wide.T @ np.linalg.inv(A_wide @ A_wide.T)
L_of_AT = R.T
print(R)
print("Wide matrix test (with big data):")
print("\tShape A:", A_wide.shape)
print("\tCheck (R^T)(A^T) ≈ I:", is_close(L_of_AT @ A_wide.T, np.eye(m)))

For small example:
	Check (R^T)(A^T) = I :
	 [[1. 0.]
	 [0. 1.]]
	Is close to identity? True
[[ 0.141079 -1.371715 -0.908259 ... -0.946482 -0.480013  0.364467]
 [ 0.898567  0.687072 -0.957172 ... -1.279203 -1.23547  -0.288179]
 [-0.888574 -0.868514 -0.274948 ... -0.843026  0.827733  0.89499 ]
 ...
 [-0.376378  2.619815  0.027312 ...  0.510293  0.494406  1.251441]
 [-0.232003 -1.898695  1.614734 ... -0.166546  1.791743 -1.219938]
 [ 0.550811 -0.288851  0.175449 ... -0.298075 -0.327788 -0.31928 ]]
(50, 2000)
[[ 0.000164  0.000432 -0.000339 ... -0.000231 -0.000231  0.000231]
 [-0.000645  0.000199 -0.000362 ...  0.001368 -0.001004 -0.000142]
 [-0.000507 -0.000352 -0.000104 ...  0.000082  0.000886 -0.000021]
 ...
 [-0.000585 -0.000656 -0.000349 ...  0.000225 -0.000045 -0.000181]
 [-0.000225 -0.00055   0.000461 ...  0.000227  0.000818 -0.000195]
 [ 0.00015  -0.000053  0.000444 ...  0.00056  -0.000498 -0.000195]]
Wide matrix test (with big data):
	Shape A: (50, 2000)
	Check (R^T)(A^T) ≈ I: Tr

$$
R = A^T (A A^T)^{-1}, 
\quad
A = \begin{bmatrix}
1 & 2 & 3 \\
2 & 1 & 0
\end{bmatrix}.
$$

---


$$
A A^T =
\begin{bmatrix}
1 & 2 & 3 \\
2 & 1 & 0
\end{bmatrix}
\begin{bmatrix}
1 & 2 \\
2 & 1 \\
3 & 0
\end{bmatrix}
=
\begin{bmatrix}
14 & 4 \\
4 & 5
\end{bmatrix}.
$$

---


$$
(A A^T)^{-1} 
= \tfrac{1}{54}
\begin{bmatrix}
5 & -4 \\
-4 & 14
\end{bmatrix}.
$$

---


$$
R =
\begin{bmatrix}
1 & 2 \\
2 & 1 \\
3 & 0
\end{bmatrix}
\cdot
\frac{1}{54}
\begin{bmatrix}
5 & -4 \\
-4 & 14
\end{bmatrix}.
$$

Compute columns separately:

* First column: $\tfrac{1}{54} [1\cdot 5 + 2(-4), \; 2\cdot 5 + 1(-4), \; 3\cdot 5 + 0]$
  $\;\;\;= \tfrac{1}{54}[-3, 6, 15] = \big[-1/18,\; 1/9,\; 5/18\big]^T.$

* Second column: $\tfrac{1}{54} [1(-4) + 2(14), \; 2(-4)+1(14), \; 3(-4)+0]$
  $\;\;\;= \tfrac{1}{54}[24, 6, -12] = \big[4/9,\; 1/9,\; -2/9\big]^T.$

So

$$
R = 
\begin{bmatrix}
-1/18 & 4/9 \\
1/9 & 1/9 \\
5/18 & -2/9
\end{bmatrix}.
$$

---


$$
A R =
\begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}.
$$



Verification: `(R^T)(A^T) ≈ I₂`.
So indeed, the transpose of the right inverse is a left inverse for $A^\top$.

### Hence,

* For **tall $A$**: compute left inverse $L$, then $L^\top$ is right inverse of $A^\top$.
* For **wide $A$**: compute right inverse $R$, then $R^\top$ is left inverse of $A^\top$.

Both exercises confirm the symmetry: the transpose trick always swaps "left inverse" and "right inverse."
